# Signal processing course 2018/2019-1 @ ELTE
# Assignment 1
## 09.17.2018

## Task 10
### Multiplicative congruential random number generator

Multiplicative congruential pseudo-random number generator, generates numbers by a recursive equation, containing 3 parameters and 1 variable, as follows:

$$
x_{i+1} = \left( a \cdot x_{i} + b \right) \mod m
$$

In [ ]:
import numpy as np
import pandas as pd
from scipy.stats import gaussian_kde

import seaborn as sns
import matplotlib.cm as cm
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits.axes_grid1.axes_divider import make_axes_locatable

In [ ]:
# Initialize seaborn with custom settings
# Facecolor values from S. Conradi @S_Conradi/@profConradi
custom_settings = {
    'figure.facecolor': '#f4f0e8',
    'axes.facecolor': '#f4f0e8',
    'axes.edgecolor': '0.7',
    'axes.linewidth' : '2',
    'grid.color': '0.7',
    'grid.linestyle': '--',
    'grid.alpha': 0.6,
}
sns.set_theme(rc=custom_settings)

#### Number of random numbers to generate

In [ ]:
N = 5000

#### Compose a multiplicative congruential RNG as described above

In [ ]:
def mult_cong_rng(a, b, m, xi):
    '''
    Implement a simple multiplicative congruential RNG.
    '''
    xp = ((a * xi + b) % m) / m
    xi = xp * m

    return xi, xp

#### Define initial state of the RNG and generate random numbers

In [ ]:
a = 13
b = 0
m = 150
x0 = 1

x_rng = np.zeros(N, dtype=np.float32)
for i in range(N):
    x0, x_rng[i] = mult_cong_rng(a, b, m, x0)

In [ ]:
fig, ax = plt.subplots(figsize=(8, 4))

sns.kdeplot(x_rng, color='0.3', fill=True, ax=ax)

ax.set_xlabel('Numbers', fontsize=14)
ax.set_ylabel('Density', fontsize=14)
ax.set_title('Fig. 1. Distribution of random numbers, by mult. cong. RNG',
             fontsize=14, y=-0.3)

plt.show()

### Marsaglia's theorem/Marsaglia effect

The Marsaglia effect encapsulates the regularity appearing, as pseudo-random numbers are generated using an iterative RNG. It was named after Marsaglia, who first observed this phenomenom.

Marsaglia showed that $n$-dimensional vectors, which were created by a pseudo-random number generator will be located on a finite number of parallel $n$-dimensional surfaces, when they're represented in a coordinate system.

We can visualize the Marsaglia effect by generating a couple thousand of $3$-dimensional vectors via the following indexing scheme:

$$
\left( x_{1}, x_{2}, x_{3} \right), \left( x_{4}, x_{5}, x_{6} \right), \left( x_{7}, x_{8}, x_{9} \right) \cdots
$$

Then we plot them on a cluster density graph, and a 2D scatter plot for demonstration.

#### Hyperparameters of the Marsaglia "simulation"

In [ ]:
N_me = 15000  # Number of vectors to generate for the Marsaglia Effect
rank = 3      # Number of components of generated vectors

#### Define initial state of the RNG and generate random numbers

In [ ]:
a = 13
b = 0
m = 150
x0 = 1

x_me = np.zeros((N_me, rank))  # Shape = (N_me x rank)
# Iterate over vectors
for i in range(N_me):
    # Iterate over components and generate random numbers for them
    for ri in range(rank):
        x0, x_me[i, ri] = mult_cong_rng(a, b, m, x0)

#### 3D scatter plot

In [ ]:
fig, ax = plt.subplots(figsize=(8, 8),
                       subplot_kw={'projection': '3d'})
ax.view_init(elev=35, azim=10)

ax.scatter(*x_me.T, color='0.3', s=4, alpha=0.4)
ax.set_xlabel('X coordinates', fontsize=14)
ax.set_ylabel('Y coordinates', fontsize=14)
ax.set_zlabel('Z coordinates', fontsize=14)

plt.show()

#### 3D density plot

In [ ]:
# Fit a Gaussian kernel on our random data
kernel = gaussian_kde(x_me.T)
# Get density values for the actual data points
density = kernel(x_me.T)

In [ ]:
fig, ax = plt.subplots(figsize=(8, 8),
                       subplot_kw={'projection': '3d'})
ax.view_init(elev=35, azim=10)

contour = ax.scatter(*x_me.T, c=density, cmap=cm.jet, s=4, alpha=0.4)
ax.set_xlabel('X coordinates', fontsize=14)
ax.set_ylabel('Y coordinates', fontsize=14)
ax.set_zlabel('Z coordinates', fontsize=14)

plt.show()

#### 2D scatter plot for each axes

In [ ]:
# Makes it easier to plot the 2D cross section plots if we encode the
# individual components as `x`, `y` and `z` using a pandas DataFrame
df = pd.DataFrame(data=x_me, columns=['x', 'y', 'z'])

In [ ]:
nr, nc = 1, 3
fig, axes = plt.subplots(nr, nc, figsize=(nc*6, nr*6))
axes = axes.flatten()

# Define axis labels
ax_labels = {
    f'{c.lower()}' : f'{c.upper()} coordinates' for c in ['x', 'y', 'z']
}
# Define the individual camera angles for all 3 plots
# Must be lists to reference pandas DataFrame keys below
views = [
    ['x', 'y'], ['y', 'z'], ['x', 'z']
]
for i, ax in enumerate(axes):
    ax.scatter(*df[views[i]].values.T,
               color='0.3', s=4, alpha=0.4)
    ax.set_xlabel(ax_labels[views[i][0]], fontsize=14)
    ax.set_ylabel(ax_labels[views[i][1]], fontsize=14)

plt.show()

#### 2D density plot for each axes

In [ ]:
nr, nc = 1, 3
fig, axes = plt.subplots(nr, nc, figsize=(nc*6, nr*6))
axes = axes.flatten()

# Define axis labels
ax_labels = {
    f'{c.lower()}' : f'{c.upper()} coordinates' for c in ['x', 'y', 'z']
}
# Define the individual camera angles for all 3 plots
# Must be lists to reference pandas DataFrame keys below
views = [
    ['x', 'y'], ['y', 'z'], ['x', 'z']
]
for i, ax in enumerate(axes):
    ax.scatter(*df[views[i]].values.T,
               c=density, cmap=cm.jet, s=4, alpha=0.4)
    ax.set_xlabel(ax_labels[views[i][0]], fontsize=14)
    ax.set_ylabel(ax_labels[views[i][1]], fontsize=14)

plt.show()